In [0]:
print("Healthy")

In [0]:
import json
from datetime import datetime, timedelta
from pyspark.sql.functions import lit
import dask
import dask.dataframe as dd
from sqlalchemy import create_engine

In [0]:
import json
from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector

import warnings
warnings.filterwarnings('ignore')

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')


Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)



Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [0]:
from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 2)).strftime('%Y-%m-%d')
last_7_days = (datetime.today() - timedelta(days = 8)).strftime('%Y-%m-%d')


print(yesterday)
print(last_7_days)


In [0]:
def load_table(schema_name, table_name, dataframe):

    chunk_size = 50000
    num_rows = len(dataframe)

    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = 'append', chunksize = 5000, method = 'multi')

    print("Data loaded successfully.")
    

In [0]:
df_max = pd.read_sql(f'''
SELECT 
MAX(transaction_date) as max_transaction_date,
MAX(transaction_id) AS max_transaction_id
FROM dwh_lux_pos_transactions
''', conn)
df_max

In [0]:
len(df_max)

In [0]:
DEFAULT_ID = 0
DEFAULT_DATE = '2000-01-01 00:00:00'

LAST_TIME =  df_max['max_transaction_date'][0] 
LAST_ID = df_max['max_transaction_id'][0] 

if LAST_ID is None:
    LAST_ID = DEFAULT_ID

if LAST_TIME is None:
    LAST_TIME = DEFAULT_DATE

print(LAST_TIME, " - ",  LAST_ID)

redshift_df = pd.read_sql_query(f'''
SELECT 
transaction_id 
FROM dwh_lux_pos_transactions
WHERE transaction_id is not null
AND transaction_id != '';
                ''', conn)
redshift_df
redshift_df = redshift_df.drop_duplicates(subset=['transaction_id'])

dump_df = pd.read_sql_query(f'''
SELECT lmt.CREDIT_RESOURCE AS 'transaction_id'
FROM LUX_MERCHANT_TRANSACTIONS lmt
WHERE lmt.CREDIT_RESOURCE IS NOT NULL
AND lmt.CREDIT_RESOURCE != '';
                ''', Dumpdb)

print(len(redshift_df), len(dump_df))

merged_df = dump_df.merge(redshift_df, how='outer', indicator=True)

# Filter out the rows that are only in the left DataFrame (df1)
missing_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the '_merge' column if not needed
missing_df = missing_df.drop(columns='_merge')

missing_df.head()

len(missing_df)

In [0]:
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)


# MIFOS_DB
mifos_driver = "org.mariadb.jdbc.Driver"
mifos_database_host = data['mifos']['host']
mifos_database_port = "3306" # update if you use a non-default port
mifos_database_name = data['mifos']['database']
mifos_user=data['mifos']['user']
mifos_password = data['mifos']['passwd']
mifos_url = f"jdbc:mysql://{mifos_database_host}:{mifos_database_port}/{mifos_database_name}"

# REDSHIFT_DWH
redshift_driver = "org.postgresql.Driver"
redshift_host = data['redshift']['host']
redshift_db_name = data['redshift']['database']
redshift_user = data['redshift']['user']
redshift_password = data['redshift']['passwd']
redshift_url = f"jdbc:postgresql://{redshift_host}:5439/{redshift_db_name}"

# DUMP_DB
dump_driver = mifos_driver
dump_host = data['Dumpdb']['host']
dump_db_name = data['Dumpdb']['database']
dump_user = data['Dumpdb']['user']
dump_password = data['Dumpdb']['passwd']
dump_url = f"jdbc:mysql://{dump_host}:3306/{dump_db_name}"


# redshift connection for dask
redshift_conn = f'postgresql://{redshift_user}:{redshift_password}@{redshift_host}:5439/{redshift_db_name}'


# Define function to extract data from dbs
def extract_function(query, url, user, password):
    df = spark.read \
                .format("jdbc") \
                .option("query", query) \
                .option("url", url) \
                .option("user", user) \
                .option("password", password) \
                .option("connectTimeout", "1000") \
                .option("treatEmptyValueAsNulls","true") \
                .option("maxRowsInMemory",200) \
                .load()
    return df


In [0]:
#AND lmt.CREDIT_RESOURCE IN ({','.join([str(x) for x in missing_df['transaction_id'].tolist()])}) 
## For Mifos
sql = f'''
SELECT 
lmt.TRANSACTION_TIME AS 'transaction_date',
lmt.TRANSACTION_TYPE AS 'transaction_type',
lmt.REFERENCE AS 'reference',
lmt.TERMINAL_ID AS 'terminal_id',
lmt.STATUS_CODE AS 'status_code',
lmt.STATUS_DESCRIPTION AS 'status_description',
'LUX' AS 'vendor',
lmt.AMOUNT AS 'amount',
lmt.SETTLED_AMOUNT AS 'settled_amount',
-- lmt.Vendor fee
-- lmt.Lux fee
lmt.PAN as card_pan,
lmt.CARD_EXPIRY AS 'card_expiry',
lmt.CARD_TYPE as 'card_type',
lmt.CARD_BANK as 'card_bank',
lmt.RRN AS 'rrn',
lmt.STAN AS 'stan',
lmt.PROCESSOR as processor,
lmt.MERCHANT_ID AS 'merchant_id',
ifnull(lmt.ACCOUNT_NUMBER, 0) as account_no,
lmt.CREDIT_RESOURCE AS 'transaction_id',
lmt.BATCH_NUMBER as 'batch_no',
LEFT(lmt.PAN, 6) AS 'first_six_digits',
RIGHT(lmt.PAN, 4) AS 'last_four_digits',
NOW() AS load_date
FROM LUX_MERCHANT_TRANSACTIONS lmt
left JOIN MM_POS_TERMINALS mpt ON mpt.MERCHANT_ID = lmt.MERCHANT_ID
where lmt.STATUS_CODE = '00'
AND lmt.CREDIT_RESOURCE > {LAST_ID} AND  lmt.TRANSACTION_TIME > '{LAST_TIME}'
group by lmt.CREDIT_RESOURCE
order by lmt.TRANSACTION_TIME
LIMIT 1000000
'''       
#                  
# apply function
dflux = extract_function(query=sql, url=dump_url, user=dump_user, password=dump_password)
dflux.persist()
dflux = dflux.toPandas()

In [0]:
dflux.head(1)

In [0]:
#lux = f'''  '''
#dflux = pd.read_sql_query(lux, Dumpdb)

dflux['account_no'] = dflux['account_no'].astype(str)

print(dflux['account_no'].nunique())
dflux.head(1)

In [0]:
%%time
accts = ','.join([str(x) for x in dflux['account_no'].unique().tolist()])
clt_name = pd.read_sql(f'''
            select 
            dac.client_id,
            dac.client_name ,
            daa.account_no,
            daa.product_id,
            daa.product_name
            from dwh_all_clients dac 
            join dwh_all_accounts daa on daa.client_id  = dac.client_id 
            where daa.account_no in ({accts})
            ''', conn)

dflw = pd.merge(dflux, clt_name, on = 'account_no', how = 'left')

dflw['charges'] = dflw['amount'].astype(float) - dflw['settled_amount'].astype(float)

dflw['amount'] = dflw['amount'].astype(float)

dflw = dflw[['transaction_date', 'transaction_type', 'reference', 'terminal_id', 'client_id', 'client_name',
   'status_code', 'status_description', 'vendor', 'amount', 'settled_amount', 'charges',
    'card_pan', 'card_expiry', 'card_type', 'card_bank', 'rrn', 'stan', 'processor', 'merchant_id',
   'account_no', 'product_id', 'product_name', 'transaction_id', 'first_six_digits', 'last_four_digits',
   'batch_no']]

In [0]:
len(dflw)

In [0]:
%%time

load_table('public','dwh_lux_pos_transactions',dflw)

#dflw.to_sql("dwh_lux_pos_transactions", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

## FAILED POS TRANSACTIONS

In [0]:
sql = f'''
SELECT 
lmt.TRANSACTION_TIME AS 'transaction_date',
lmt.TRANSACTION_TYPE AS 'transaction_type',
lmt.REFERENCE AS 'reference',
lmt.TERMINAL_ID AS 'terminal_id',
lmt.STATUS_CODE AS 'status_code',
lmt.STATUS_DESCRIPTION AS 'status_description',
'LUX' AS 'vendor',
lmt.AMOUNT AS 'amount',
lmt.SETTLED_AMOUNT AS 'settled_amount',
-- lmt.Vendor fee
-- lmt.Lux fee
lmt.PAN as card_pan,
lmt.CARD_EXPIRY AS 'card_expiry',
lmt.CARD_TYPE as 'card_type',
lmt.CARD_BANK as 'card_bank',
lmt.RRN AS 'rrn',
lmt.STAN AS 'stan',
lmt.PROCESSOR as processor,
lmt.MERCHANT_ID AS 'merchant_id',
ifnull(lmt.ACCOUNT_NUMBER, 0) as account_no,
lmt.CREDIT_RESOURCE AS 'transaction_id',
lmt.BATCH_NUMBER as 'batch_no',
LEFT(lmt.PAN, 6) AS 'first_six_digits',
RIGHT(lmt.PAN, 4) AS 'last_four_digits',
NOW() AS load_date
FROM LUX_MERCHANT_TRANSACTIONS lmt
left JOIN MM_POS_TERMINALS mpt ON mpt.MERCHANT_ID = lmt.MERCHANT_ID
where lmt.STATUS_CODE != '00'
AND LEFT(lmt.TRANSACTION_TIME,10) >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)
order by lmt.TRANSACTION_TIME
'''                    
# apply function
dflux = extract_function(query=sql, url=dump_url, user=dump_user, password=dump_password)
dflux.persist()
dflux = dflux.toPandas()

dflux['account_no'] = dflux['account_no'].astype(str)
len(dflux)

In [0]:
dflux.head()

In [0]:


accts = ','.join([str(x) for x in dflux['account_no'].unique().tolist()])
clt_name = pd.read_sql(f'''
            select 
            dac.client_id,
            dac.client_name ,
            daa.account_no,
            daa.product_id,
            daa.product_name
            from dwh_all_clients dac 
            join dwh_all_accounts daa on daa.client_id  = dac.client_id 
            where daa.account_no in ({accts})
            ''', conn)

dflw = pd.merge(dflux, clt_name, on = 'account_no', how = 'left')

dflw['charges'] = dflw['amount'].astype(float) - dflw['settled_amount'].astype(float)

dflw['amount'] = dflw['amount'].astype(float)

dflw = dflw[['transaction_date', 'transaction_type', 'reference', 'terminal_id', 'client_id', 'client_name',
   'status_code', 'status_description', 'vendor', 'amount', 'settled_amount', 'charges',
    'card_pan', 'card_expiry', 'card_type', 'card_bank', 'rrn', 'stan', 'processor', 'merchant_id',
   'account_no', 'product_id', 'product_name', 'transaction_id', 'first_six_digits', 'last_four_digits',
   'batch_no']]
dflw.head()

In [0]:
len(dflw)

In [0]:
def load_table(schema_name, table_name, dataframe, if_exists_action ):

    chunk_size = 10000
    num_rows = len(dataframe)

    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = if_exists_action, chunksize = 5000, method = 'multi')

    print("Data loaded successfully.")
    

In [0]:
%%time

load_table('public','dwh_lux_pos_transactions_failed',dflw, 'replace')

## Vpay pos ETL

In [0]:
vplast = pd.read_sql_query(f'''
                select
                max(id) as last_tid,
                max(transaction_date) as last_date
                from dwh_vpay_pos_transactions
                ''', conn)
vtr = vplast['last_tid'][0], vplast['last_date'][0]

vtr[0], vtr[1]

In [0]:
%%time

## Vpay pos transactions 
vp = f'''
SELECT 
tw.id,
tw.TRANSACTION_ID as transaction_reference,
tw.STATEMENT_IDS as transaction_id,
tw.WALLET_NAME as partner,
tw.TRANSACTION_TYPE as transaction_type,
tw.TRANSACTION_RESPONSE as status_code,
tw.FROM_ACCOUNT_NO as 3rd_party_account,
tw.FROM_BANK as 3rd_party_bank_code,
al.bank_name,
tw.TO_ACCOUNT_NO as account_no,
tw.AMOUNT as amount,
tw.TIME as transaction_date
FROM TM_WALLET_TRANSACTIONS tw
left join AM_LOCAL_BANKS al on al.NIP_CODE = tw.FROM_BANK
WHERE tw.WALLET_NAME LIKE '%vpos%'
and tw.id > {vtr[0]}
'''

dfvp = pd.read_sql_query(vp, Dumpdb)

dfvp

In [0]:
%%time
def get_account_details(account_no_list):
    accts = ','.join([str(x) for x in account_no_list])

  
    mif = f'''select replace(mc.display_name, 'null', '') as client_name, sa.account_no, sa.id, sa.product_id, sa.account_balance_derived, sa.approvedon_date, mc.referred_by_id
    from m_client mc join m_savings_account sa on sa.client_id = mc.id 
    where sa.account_no in ({accts})
    '''
    try:
        dfmif = pd.read_sql_query(mif, Mifosdb)
        
        output =  dfmif

    except Exception as e:
        
        print(e)
        
        output = (f'''An error occured! Error details - {e}.
        NOTE - This function only takes list as argument.''')
                
    return output
        
                                                                                                        
if len(dfvp) > 0:
    dfcl = get_account_details(dfvp['account_no'].tolist())[['client_name', 'account_no']]
    full = pd.merge(dfvp, dfcl, on = 'account_no', how = 'left')
    
    full.to_sql('dwh_vpay_pos_transactions', conn, index = False, if_exists = 'append', chunksize = 10000, method = 'multi')
    
else:
    print("No transactions to load at this time")

In [0]:
%%time
## 
df_ter = f'''
SELECT 
p.id,
mc.display_name AS client_name, 
msa.account_no, 
p.TERMINAL_ID AS terminal_id, 
p.savings_account_id,
msp.name AS savings_account_product,
p.MSC_PERCENT,
p.DELAYED_SETTLEMENT,
p.BULK_SETTLEMENT,
p.DELAYED_HOUR_TIME,
p.OWNED_BY_VFD,
p.TRF_CHARGE,
p.TRF_CAP,
p.payment_type,
NOW() AS load_date
FROM POS_TERMINALS p
LEFT JOIN m_savings_account msa ON msa.id = p.savings_account_id
LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
LEFT JOIN m_client mc ON mc.id = msa.client_id
WHERE p.id > 1
ORDER BY 1;
'''
df_ter = pd.read_sql_query(df_ter, Mifosdb)
df_ter.head()

In [0]:
def load_table(schema_name, table_name, dataframe, if_exists_action ):

    chunk_size = 10000
    num_rows = len(dataframe)

    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = if_exists_action, chunksize = 5000, method = 'multi')

    print("Data loaded successfully.")
    

In [0]:
load_table('public', 'dwh_pos_customers', df_ter, 'replace')

In [0]:
load_table('public', 'dwh_pos_categories', df_ter, 'replace')

In [0]:
import datetime
last_run_date = datetime.datetime.now()

print("Last run date:", last_run_date)